In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import boto3
import pandas as pd
import urllib

#setup
CREDENTIALS_FILE = './jessica_credentials.csv'
credentials = pd.read_csv(CREDENTIALS_FILE).to_dict('records')[0]
aws_access_key_id = credentials['Access key ID']
aws_secret_access_key = credentials['Secret access key']

region_name = 'us-east-1'
endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'

client = boto3.client(
    'mturk',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    endpoint_url = endpoint_url,
    region_name = region_name
)


In [ ]:
#create hit type
one_minute = 60 # seconds
one_hour = 60 * one_minute
one_day = 24 * one_hour

#remember to change title!!!
hit_type_response = client.create_hit_type(
    AutoApprovalDelayInSeconds = 1 * one_hour,
    AssignmentDurationInSeconds = 30 * one_minute,
    Reward = '0.1',
    Title = 'Detect emotion in very short videos (Dominance)',
    Keywords = 'emotion, GIF, detection, VAD, short videos',
    Description = 'Choose the short video that is best associated with the following words',
    QualificationRequirements = [
      {
          'QualificationTypeId': '00000000000000000071',
          'Comparator': 'In',
          'LocaleValues': [
            {
                'Country': 'TW'
            },
            {
                'Country': 'US'
            },
            {
                'Country': 'IN'
            },
          ],
          'RequiredToPreview': True,
          'ActionsGuarded': 'PreviewAndAccept'
      },
    ]
)

In [ ]:
#hit for vad
#micro pilot
#remember to change dim and 10/100!!!

hit_type_id = hit_type_response['HITTypeId']

dim = 'd'

with open("filenames.txt.tuples", "r") as fin:
  lines = fin.readlines()
  for l in lines:
    names = l.split('	')
    names[3] = names[3].replace("\n", "")

    params =  'dim=' + urllib.parse.quote(dim) + '&amp;' 'gifA=' + urllib.parse.quote(names[0]) + '&amp;' 'gifB=' + urllib.parse.quote(names[1]) + '&amp;''gifC=' + urllib.parse.quote(names[2]) + '&amp;''gifD=' + urllib.parse.quote(names[3]) 

    question=f'''<?xml version="1.0" encoding="UTF-8"?>
        <ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
        <ExternalURL>https://doraemon.iis.sinica.edu.tw/gif-vad/test/templates/micro.html?{params}</ExternalURL>
        <FrameHeight>800</FrameHeight>
        </ExternalQuestion>'''

    response = client.create_hit_with_hit_type(
          HITTypeId = hit_type_id,
          MaxAssignments = 2,
          LifetimeInSeconds = 3 * one_day,
          Question = question,
          RequesterAnnotation = dim
      )
print("done")

done


In [ ]:
#fetch results

import re
import csv

hits_paginator = client.get_paginator('list_hits')
assignments_paginator = client.get_paginator('list_assignments_for_hit')
ans = ""

v_tuples = []
a_tuples = []
d_tuples = []

vf = open("v_result.csv", "w")
vwriter = csv.writer(vf)
vwriter.writerow(['Item1','Item2','Item3','Item4','BestItem','WorstItem'])

af = open("a_result.csv", "w")
awriter = csv.writer(af)
awriter.writerow(['Item1','Item2','Item3','Item4','BestItem','WorstItem'])

df = open("d_result.csv", "w")
dwriter = csv.writer(df)
dwriter.writerow(['Item1','Item2','Item3','Item4','BestItem','WorstItem'])

for hits in hits_paginator.paginate():
  for hit in hits['HITs']:
    for assignments in assignments_paginator.paginate(HITId=hit['HITId']):
      for assignment in assignments['Assignments']:
        result = re.findall(r'<FreeText>([^<]*)</FreeText>', assignment['Answer'])
        tpl = result[3].split("_")
        tpl.append(result[1])
        tpl.append(result[0])
        if(result[2] == "v"):
          vwriter.writerow(tpl)
        elif(result[2] == "a"):
          awriter.writerow(tpl)
        elif(result[2] == "d"):
          dwriter.writerow(tpl)
  
vf.close()
af.close()
df.close()